# Generate word2vec model - include all new topic words

In [1]:
from gensim.models import Word2Vec, KeyedVectors
import gensim
import time
import numpy as np
from tqdm import tqdm
from pprint import pprint
import pickle
import spacy
import re
import operator
import spacy_udpipe

In [2]:
spacy_udpipe.download('cs')
nlp = spacy_udpipe.load('cs')

Already downloaded a model for the 'cs' language


In [3]:
# set variables 
LANG = 'cs'
MAX_WORDS = 500000  # maximum processed words to small model 

MODEL_DIR = '/mnt/data/data/models-fasttext/'
TOPICS_DIR = '/home/dzon/kajo/topics/topic_lists-based_on_en-US/'

model_filename = MODEL_DIR + 'cc.cs.300-lemma_500k_min2.vec.bin'

excluded_model_name = MODEL_DIR + 'cc.cs.300-lemma_500k_min2_excluded.vec.bin'

model_orig_filename = MODEL_DIR + 'cc.cs.300.vec.bin'
model_orig_filename_vec = MODEL_DIR + 'cc.cs.300.vec'

pickled_words_lemmas = MODEL_DIR + LANG + '_ft_all_words_lemmas.pickle'

lemma_list_file = TOPICS_DIR + 'topics_lemma_'+ LANG +'.lst'
nolemma_list_file = TOPICS_DIR + 'topics_'+ LANG +'.lst'

excluded_filename = TOPICS_DIR + 'topics_' + LANG + '-not_in_model.lst'

lemma_list_file_excluded = TOPICS_DIR + 'topics_lemma_excluded_' + LANG + '.lst'
nolemma_list_file_excluded = TOPICS_DIR + 'topics_excluded_' + LANG + '.lst'


In [4]:
# load the big model
model_big = KeyedVectors.load_word2vec_format(model_orig_filename, binary=True)

In [5]:
# process all words in fasttext CS model  takes ~12min
all_words_lemmas = []

lemmas = {}
word_lemma = {}
words = model_big.index2word
for word in tqdm(words):
    # skip "words" which are not regular words
    if not re.fullmatch("([^\d\W])+", word):
        continue
    # strange issue with CS model in version 2.5 
    if word.lower() == 'aby':
        lemma = 'aby'
    else:    
        doc = nlp(word)
        lemma = doc[0].lemma_.lower()
    all_words_lemmas.append((word, lemma))
print("Number of words in big model: ", len(all_words_lemmas))

#save pickled dump 
with open(pickled_words_lemmas, 'wb') as f:
    pickle.dump(all_words_lemmas, f)

100%|██████████| 2000000/2000000 [13:30<00:00, 2467.75it/s]


Number of words in big model:  1635266


In [6]:
#load pickled lemma list
with open(pickled_words_lemmas, 'rb') as f:
    all_words_lemmas = pickle.load(f)
pprint(all_words_lemmas[:50])

[('a', 'a'),
 ('v', 'v'),
 ('na', 'na'),
 ('se', 'se'),
 ('je', 'být'),
 ('s', 's'),
 ('z', 'z'),
 ('pro', 'pro'),
 ('do', 'do'),
 ('o', 'o'),
 ('to', 'ten'),
 ('že', 'že'),
 ('i', 'i'),
 ('ve', 'v'),
 ('k', 'k'),
 ('si', 'se'),
 ('V', 'v'),
 ('za', 'za'),
 ('od', 'od'),
 ('nebo', 'nebo'),
 ('ale', 'ale'),
 ('jako', 'jako'),
 ('jsou', 'být'),
 ('tak', 'tak'),
 ('Kč', 'kč'),
 ('po', 'po'),
 ('Více', 'hodně'),
 ('které', 'který'),
 ('byl', 'být'),
 ('jsem', 'být'),
 ('u', 'u'),
 ('roce', 'rok'),
 ('by', 'být'),
 ('který', 'který'),
 ('byla', 'být'),
 ('align', 'align'),
 ('right', 'righen'),
 ('roku', 'rok'),
 ('ze', 'z'),
 ('také', 'také'),
 ('A', 'a'),
 ('jeho', 'jeho'),
 ('až', 'až'),
 ('při', 'při'),
 ('Na', 'na'),
 ('co', 'co'),
 ('bude', 'být'),
 ('má', 'mít'),
 ('jen', 'jen'),
 ('jak', 'jak')]


In [7]:
# generate helpful dictionaries/lists
big_lemmas_set = set([l for (_,l) in all_words_lemmas])
print(len(big_lemmas_set))

big_words_set = set([w for (w,_) in all_words_lemmas])
print(len(big_words_set))

lemmas = {}
words_lower_orig = {}
big_lemma_word_dic = {}
big_word_lemma_dic = {}
for (w,l) in all_words_lemmas:
    big_word_lemma_dic[w] = l
    
    #dictionary of lowercase word -> all original words
    if w.lower() in words_lower_orig:
        words_lower_orig[w.lower()].append(w)
    else:
        words_lower_orig[w.lower()] = [w]
        
    if l in big_lemma_word_dic:
        big_lemma_word_dic[l].append(w)
        lemmas[l] += 1
    else:
        big_lemma_word_dic[l] = [w]
        lemmas[l] = 1
        
# load all the words from big model
words = model_big.wv.index2word
words_lower = [w.lower() for w in words]

#words_lemma = model_lemma_big.index2word       

856531
1635266


/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [8]:
# sort the lemmas from most frequent
sorted_lemmas = sorted(lemmas.items(), key=operator.itemgetter(1), reverse=True)
# trying first words
sorted_lemmas[:20]

[('být', 169),
 ('mít', 127),
 ('stát', 124),
 ('stránka', 120),
 ('dát', 111),
 ('jít', 103),
 ('odpovědět', 99),
 ('on', 89),
 ('malý', 89),
 ('dobrý', 87),
 ('den', 86),
 ('velký', 85),
 ('kčkoupit', 85),
 ('chtít', 83),
 ('hrát', 81),
 ('jet', 81),
 ('moci', 80),
 ('vysoký', 80),
 ('příslušenství', 80),
 ('oblíbený', 80)]

In [9]:
# create vectors for the small models - average vectors for lemmas
# process only first MAX_WORDS in the big model
tmp_words = [None] * len(model_big.wv.index2word)
tmp_vecs = [None] * len(model_big.wv.index2word) 
for i,(word,lemma) in enumerate(all_words_lemmas[:MAX_WORDS]):
    lemma_words_list = big_lemma_word_dic[lemma]
    # add only lemmas made from at least 2 words
    if len(lemma_words_list)<2:
        continue
    tmp_vectors = []
    tmp_indexes = []
    error_with_word = False
    for word2 in lemma_words_list:
        try:
            tmp_vectors.append(model_big[word2])  
            tmp_indexes.append(model_big.vocab[word2].index)
        except:
            print(i, "error with ", word, " lemma ", lemma, "words", lemma_words_list)
            error_with_word = True
            continue
    if not error_with_word:
        try:
            tmp_words[min(tmp_indexes)] = lemma
            tmp_vecs[min(tmp_indexes)] = list(np.average(tmp_vectors, axis=0))
        except:
            print(i, "error with lemma ", lemma, " - words", lemma_words_list, " indexes: ", tmp_indexes)
            # note - we are processing first MAX_WORDS from the model after the filter out of the "non-words"
            # so some of the last words in that range may refer to words on bigger index than MAX_WORDS

# filter the lists (some positions remained None)
final_words = [i for i in tmp_words if i] 
final_vecs = [i for i in tmp_vecs if i != None] 

print(len(final_vecs), len(final_words))
len(final_vecs[0])  

del tmp_vecs
del tmp_words

/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


144254 144254


In [10]:
# create new gensim word2vec model with the vectors:
new_vector_space = gensim.models.keyedvectors.Word2VecKeyedVectors(300)
new_vector_space.add(final_words,final_vecs)

In [11]:
# test the model
pprint(new_vector_space.most_similar('otec'))
pprint(new_vector_space.most_similar_cosmul(positive=['žena', 'král'], negative=['muž']))

[('syn', 0.8043177127838135),
 ('matka', 0.7384902834892273),
 ('bratr', 0.6473388671875),
 ('bůh', 0.622814953327179),
 ('tatínek', 0.6192840933799744),
 ('svatý', 0.6160073280334473),
 ('kněz', 0.6051469445228577),
 ('dcera', 0.5990678071975708),
 ('ježíš', 0.5902522802352905),
 ('boží', 0.5883111953735352)]
[('královna', 0.9381455183029175),
 ('princezna', 0.8582150936126709),
 ('království', 0.8572831153869629),
 ('královský', 0.8456887602806091),
 ('princ', 0.8358922004699707),
 ('panovník', 0.8302497267723083),
 ('královně', 0.8191059827804565),
 ('panovnice', 0.8184678554534912),
 ('kralevice', 0.8177275061607361),
 ('korunovace', 0.8090215921401978)]


In [12]:
# save the model
new_vector_space.save_word2vec_format(model_filename, binary=True)

In [13]:
#model = KeyedVectors.load_word2vec_format(model_filename, binary=True)
model = new_vector_space
words_model = model.wv.index2word

/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
# load topic listst
with open(nolemma_list_file) as fp:
    topics_nolemma = fp.readlines() 
print(len(topics_nolemma))
    
# load the lemma list
# with open(lemma_list_file) as fp:
#    topics = fp.readlines()
# print(len(topics))

topics = []
# generate the lemma list
for topic in tqdm(topics_nolemma):
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []
        
    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics.append("_".join(lemmas))
    
print(len(topics))    
    
#write the lemma topic list
with open(lemma_list_file, 'wt', encoding='utf8') as f:
    for topic_lemma in topics:
        f.write(topic_lemma + "\n")    

  1%|▏         | 66/5206 [00:00<00:07, 658.84it/s]

5206


100%|██████████| 5206/5206 [00:05<00:00, 899.05it/s] 

5206


In [15]:
#try if the topic-lemma words exist in the models  

words_to_manual_add = {}

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic = topic.strip()
    topic_words = topic.split('_')
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    for word, word_nolemma in zip(topic_words, nolemmas):
        #print(word)
        found = 0
        if word in words_model:
            found = 1
            #print("l1")
        elif word in big_lemmas_set:
            found = 1
            print("l2", word)
        elif word in words_lower:
            found = 1
            print("l3", word)
        # nolemma words are in PL case probably not needed and all lemmas exist in lemma list
        elif word_nolemma in words_model:
            found = 1
            print("w1")
        elif word_nolemma in big_lemmas_set:
            found = 1
            print("w2")
        elif word_nolemma in words_lower:
            found = 1
            print("w3")

        if found == 0:
            print(i, 'needs to be added manually: ', word, " - ", word_nolemma, " - ", topic_nolemma.strip())
            if word not in words_to_manual_add:
                words_to_manual_add[word] = topic_nolemma.strip()
                    
print(len(words_to_manual_add))          

w1
l2 územný
l2 územný
l2 venkovská
l2 plnoprávní
l2 podnebit
l2 územný
l2 hornatat
l2 průsakový
l2 venkovská
l2 cementárenský
l2 územný
426 needs to be added manually:  bezzávlahový  -  bezzávlahové  -  hospodaření bezzávlahové
l2 znovuvyužití
692 needs to be added manually:  korozivita  -  korozivita  -  korozivita vody
l2 územný
l2 břežet
l2 dehet
l2 demineralizace
w1
l2 výchovná
l2 půdoznalství
925 needs to be added manually:  sktruktura  -  sktruktura  -  sktruktura krajiny
l2 územný
l3 zneužívání
w1
w1
l2 podnebit
l2 pramenitat
l2 souvrátit
w3
l2 podnební
w1
l2 dnový
l2 směmovna
1382 needs to be added manually:  termoselektivní  -  termoselektivní  -  proces termoselektivní
l2 vyvádějící
l2 pozadí
l2 podnebit
l2 dehet
w3
l2 obydlení
l2 přetržitý
w1
1504 needs to be added manually:  protilavinový  -  protilavinové  -  opatření protilavinové
l2 vysokoproteinový
l2 chův
l2 bentický
l2 jednogenerační
l2 biologická
l2 imiset
l2 iucn
w3
l2 oddílný
l2 oddílný
l2 potrubný
w1
w1
l2 mangle

5178 needs to be added manually:  lysimetrie  -  lysimetrie  -  lysimetrie
l2 metainformace
5180 needs to be added manually:  mikroekosystém  -  mikroekosystém  -  mikroekosystém
5182 needs to be added manually:  mikroznečišťující  -  mikroznečišťující  -  látka mikroznečišťující
5186 needs to be added manually:  peroxyacetyldusičnan  -  peroxyacetyldusičnan  -  peroxyacetyldusičnan
l2 fytomaso
5189 needs to be added manually:  ploskochodý  -  ploskochodý  -  živočich ploskochodý
l2 dibenzofuran
5191 needs to be added manually:  autoekologie  -  autoekologie  -  autoekologie
l2 cévnatat
l2 unced
l2 bioreaktor
63


In [16]:
# for manual correcting of words
pprint(words_to_manual_add)

{'adsorbující': 'látka adsorbující ropu',
 'aerobiologie': 'aerobiologie',
 'agroprůmysl': 'agroprůmysl',
 'alkylický': 'sloučenina alkylická',
 'artropoda': 'Artropoda',
 'autoekologie': 'autoekologie',
 'bastardizace': 'bastardizace fauny',
 'bezzávlahový': 'hospodaření bezzávlahové',
 'chloretylna': 'chloretylen',
 'chlorfenol': 'chlorfenol',
 'chlorfluorovaný': 'uhlovodík chlorfluorovaný',
 'coelenterate': 'bezobratlí Coelenterate',
 'dezintegrátor': 'dezintegrátor šrédr',
 'edafologie': 'edafologie',
 'embryogenéza': 'embryogenéza',
 'flokuace': 'flokuace',
 'fluidika': 'fluidika',
 'fluidizovaný': 'lože fluidizované',
 'fytosociologie': 'fytosociologie',
 'geogenní': 'faktor geogenní',
 'georeferenční': 'informace georeferenční',
 'geotechnologie': 'geotechnologie',
 'haloform': 'haloform',
 'heteroctera': 'hmyz Heteroctera',
 'imunotest': 'imunotest',
 'jevnosnubný': 'rostlina jevnosnubná',
 'korozivita': 'korozivita vody',
 'lipofilický': 'látka lipofilická',
 'lysimetrie': 'ly

In [17]:
# the lemmas not in model were manually checked if they exist in other models in some variation
word = "antarktyki"
print(word)
try:
    model[word]
    print('model')
    pprint(model.most_similar(word))
except:
    pass

if word in big_lemmas_set:
    print('model_lemma_big', big_lemma_word_dic[word])

try:
    model_big[word]
    print('model big')
    pprint(model_big.most_similar(word))
except:
    pass

# if the original word with uppercase
if word in words_lower_orig:
    print('model big LOWER', words_lower_orig[word])
    pprint(model_big.most_similar(words_lower_orig[word]))
    

antarktyki


In [18]:
# for cases the lemmas aren't correct or don't exist in the dic
# manually made dictionary of "lemma" : "word which exists in some model" or "word1;word2" for the average
# e.g. 'mikroekosystém': 'mikro;ekosystém', 'bahný': 'bahno'
new_words_manually_checked = {'AIDS': 'aids',
 'adsorbující': 'absorbující',
 'aerobiologie': 'aero;biologie',
 'agroekosystém': 'agro;ekosystém',
 'agroprůmysl': 'agro;průmysl',
 'akumulak': 'akumulace',
 ###'alkylický': 'sloučenina alkylická',
 'alveola': 'alveola',
 'amalgat': 'amalgám',
 ###'artropoda': 'Artropoda',
 'astronautika': 'astronautika',
 'atrazina': 'atrazin',
 'autoekologie': 'auto;ekologie',
 'bahný': 'bahno',
 ###'bastardizace': 'bastardizace fauny',
 ###'bentický': 'ekosystém bentický',
 'benzený': 'benzen',
 'benzopyrna': 'benzopyren',
 'bezjaderný': 'bezjaderný',
 'bezobratlovec': 'bezobratlovec',
 'bezzávlahový': 'bez;závlahový',
 'břežet': 'pobřeží',
 ###'chloretylna': 'chloretylen',
 ###'chlorfenol': 'chlorfenol',
 ###'chlorfluorovaný': 'uhlovodík chlorfluorovaný',
 ###'coelenterate': 'bezobratlí [Coelenterate]',
 'cévnatat': 'cévnatá',
 ###'dezintegrátor': 'dezintegrátor [šrédr]',
 ###'dibenzodioxin': 'dibenzodioxin polychlorovaný',
 ###'dibenzofuran': 'dibenzofuran polychlorovaný',
 'dložet': 'podloží',
 ###'edafologie': 'edafologie',
 'ekotoxikologický': 'ekotoxikologický',
 ###'elektrokinetika': 'elektrokinetika',
 ###'embryogenéza': 'embryogenéza',
 'emiseň': 'emisní',
 'evidenec': 'evidence',
 ###'flokuace': 'flokuace',
 ###'fluidika': 'fluidika',
 ###'fluidizovaný': 'lože fluidizované',
 'fluóra': 'fluór',
 'fytomaso': 'fytomasa',
 'fytosociologie': 'fyto;sociologie',
 'fósforum': 'fosfor',
 ###'geogenní': 'faktor geogenní',
 'georeferenční': 'georeference',
 'geotechnologie': 'geo;technologie',
 'globálný': 'globální',
 ###'haloform': 'haloform',
 'havarijný': 'havarijní',
 ###'heteroctera': 'hmyz [Heteroctera]',
 'hornatat': 'hornatý',
 'hygieno': 'hygiena',
 'imiset': 'imise',
 'imunotest': 'imuno;test',
 'izolačný': 'izolační',
 ###'jevnosnubný': 'rostlina jevnosnubná',
 'koev': 'kov',
 ###'komplexotvorný': 'činidlo komplexotvorné',
 'kopytnatec': 'kopytnatec',
 'korozivita': 'korodovat',
 'krisový': 'krizový',
 'kyanat': 'kyanat',
 'leptace': 'leptat',
 'lignita': 'lignit',
 'lihovarský': 'lihovarský',
 ###'lipofilický': 'látka lipofilická',
 ###'lysimetrie': 'lysimetrie',
 ###'manglový': 'porost manglový pobřežní',
 ###'metanizace': 'metanizace',
 'mikroekosystém': 'mikro;ekosystém',
 'mikroklimatologie': 'mikro;klimatologie',
 'mikroznečišťující': 'mikro;znečišťující',
 ###'mláze': 'les smíšený s mlázím',
 ###'mlázet': 'mlází',
 'mořicí': 'mořící',
 ###'myceta': 'mycety',
 'místopřísežný': 'místopřísežný',
 'naplaveninový': 'naplavenina',
 ###'nitrosamín': 'nitrosamin',
 'obrazův': 'obrazový',
 ###'oceanografie': 'oceanografie v ústí řeky',
 'oceaný': 'oceán',
 ###'odkrývka': 'odkrývka',
 'ojetat': 'ojetý',
 ###'olfaktometrie': 'olfaktometrie',
 ###'perchloretylna': 'perchloretylen',
 ###'perifyton': 'perifyton',
 ###'peroxyacetyldusičnan': 'peroxyacetyldusičnan',
 ###'petrochemikálie': 'petrochemikálie',
 ###'ploskochodý': 'živočich ploskochodý',
 'podnebit': 'podnebí',
 ###'policý': 'bubble policy [povolení obchodovatelná emisní]',
 ###'polymro': 'polymer',
 'potrubný': 'potrubní',
 'pramenitat': 'pramenitá',
 ###'prekluha': 'prekluze',
 'protilavinový': 'proti;lavina',
 'průsako': 'průsak',
 ###'pyralen': 'pyralen',
 'přístřešit': 'přístřeší',
 ###'reflektometrie': 'reflektometrie',
 ###'remobilizace': 'remobilizace látky znečišťující',
 ###'resuspenzace': 'resuspenzace',
 'roponosný': 'roponosný',
 'rozmanitat': 'rozmanitý',
 'rozprašovaný': 'rozprašovaný',
 ###'salinizace': 'salinizace',
 'sktruktura': 'struktura',
 'směmovna': 'sněmovna',
 'sociopolitický': 'sociopolitický',
 ###'solitvorný': 'oblast solitvorná',
 'souvrátit': 'souvrať',
 'syndromě': 'syndrom',
 'sůt': 'sůl',
 ###'tajnosnubný': 'rostlina tajnosnubná',
 ###'tenatový': 'lov ryb do sítí tenatových',
 ###'teratogeneze': 'test teratogeneze',
 ###'terfenyl': 'terfenyl halogenovaný',
 ###'termoselektivní': 'proces termoselektivní',
 ###'triazina': 'triazin',
 'těkavat': 'netěkavý',
 'urotoxicita': 'neurotoxicita',
 'velkorozměrný': 'velkorozměrný',
 'venkův': 'venkov',
 ###'vináza': 'vináza',
 ###'vitirifikace': 'vitirifikace',
 'vodonosný': 'vodonosný',
 'vymáhánit': 'vymáhání',
 'vúpravo': 'doúprava',
 ###'xenobiotický': 'látka xenobiotická',
 'zemín': 'zemina',
 'zneškodnitelný': 'zneškodnit',
 'závazel': 'závazek',
 'étro': 'éter',
 'ústrojit': 'dýchací',
 'územný': 'územní',
 ###'článkovec': 'článkovec',
 ###'šagasův': 'nemoc Šagasova',
 'škůdec': 'škůdci',
 ###'šrédr': 'dezintegrátor [šrédr]',
 'štěrkopísko': 'štěrkopísek'}
    
len(new_words_manually_checked)

83

In [19]:
# function which finds the word in all the available models and returns it's vector (average vector)
def get_vector(word):
    # word is in small model
    try:
        return model[word]
    except:
        pass
    
    # word is in lemmas from big model - return average if the lemma was result of more words
    if word in big_lemmas_set:
        lemma_words_list = big_lemma_word_dic[word]
        tmp_vectors = []
        for word2 in lemma_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

    try:
        return model_big[word]
    except:
        pass
    
    # if the original word with uppercase
    if word in words_lower_orig:
        orig_words_list = words_lower_orig[word]
        tmp_vectors = []
        for word2 in orig_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

        #return model_big[words_lower_orig[word]]
    
    return []

In [20]:
# ading words from manually checked list
new_vectors = []
new_words = []
for i, word in enumerate(new_words_manually_checked.keys()):
    tmp_vectors = []
    print(i, word)
    for tmp_word in new_words_manually_checked[word].split(';'):
        vector = get_vector(tmp_word)
        if vector != []:
            tmp_vectors.append(vector)
        else:
            print("XXX - some ERROR, word doesn't have vector, cannot add: ", tmp_word)
    if word not in new_words:
        new_vectors.append(np.average(tmp_vectors, axis=0))
        new_words.append(word)

0 AIDS
1 adsorbující
2 aerobiologie
3 agroekosystém
4 agroprůmysl
5 akumulak
6 alveola
7 amalgat
8 astronautika
9 atrazina
10 autoekologie
11 bahný
12 benzený
13 benzopyrna
14 bezjaderný
15 bezobratlovec
16 bezzávlahový
17 břežet
18 cévnatat
19 dložet
20 ekotoxikologický
21 emiseň
22 evidenec
23 fluóra
24 fytomaso
25 fytosociologie
26 fósforum
27 georeferenční
28 geotechnologie
29 globálný
30 havarijný
31 hornatat
32 hygieno
33 imiset
34 imunotest
35 izolačný
36 koev
37 kopytnatec
38 korozivita
39 krisový
40 kyanat
41 leptace
42 lignita
43 lihovarský
44 mikroekosystém
45 mikroklimatologie
46 mikroznečišťující
47 mořicí
48 místopřísežný
49 naplaveninový
50 obrazův
51 oceaný
52 ojetat
53 podnebit
54 potrubný
55 pramenitat
56 protilavinový
57 průsako
58 přístřešit
59 roponosný
60 rozmanitat
61 rozprašovaný
62 sktruktura
63 směmovna
64 sociopolitický
65 souvrátit
66 syndromě
67 sůt
68 těkavat
69 urotoxicita
70 velkorozměrný
71 venkův
72 vodonosný
73 vymáhánit
74 vúpravo
75 zemín
76 zneškod

/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


In [21]:
# add new words to the model
model.add(new_words,new_vectors)

In [22]:
# 2nd run - checking if there are still unavailable words in topics
excluded_topics = []
#try if the topic-lemma words exist in the model   

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    topic = topic.strip()
    topic_nolemma = topic_nolemma.strip()
    topic_words = topic.split('_')
    found = 1
    for word in topic_words:
        #print(word)
       
        vector = get_vector(word)
        if vector == []:
            found = 0
            print('nonexistend word: ', word, " - ", topic_nolemma)

    if found == 0:
        print('!!! topic needs to be excluded: ', topic_nolemma.strip())
        if topic_nolemma.strip() not in excluded_topics:
            excluded_topics.append(topic_nolemma.strip())
                    
print(len(excluded_topics)) 

nonexistend word:  termoselektivní  -  proces termoselektivní
!!! topic needs to be excluded:  proces termoselektivní
nonexistend word:  policý  -  bubble policy [povolení obchodovatelná emisní]
!!! topic needs to be excluded:  bubble policy [povolení obchodovatelná emisní]
nonexistend word:  oceanografie  -  oceanografie v ústí řeky
!!! topic needs to be excluded:  oceanografie v ústí řeky
nonexistend word:  salinizace  -  salinizace
!!! topic needs to be excluded:  salinizace
nonexistend word:  bastardizace  -  bastardizace fauny
!!! topic needs to be excluded:  bastardizace fauny
nonexistend word:  manglový  -  porost manglový pobřežní
!!! topic needs to be excluded:  porost manglový pobřežní
nonexistend word:  resuspenzace  -  resuspenzace
!!! topic needs to be excluded:  resuspenzace
nonexistend word:  petrochemikálie  -  petrochemikálie
!!! topic needs to be excluded:  petrochemikálie
nonexistend word:  mláze  -  les smíšený s mlázím
!!! topic needs to be excluded:  les smíšený s

/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [23]:
# save topics containing words which are not present neither model - to be excluded from the topic list
print('excluded topics: ', len(excluded_topics))
pprint(excluded_topics)
with open(excluded_filename, 'wt', encoding='utf8') as f:
    for topic in excluded_topics:
        f.write(topic + "\n")

excluded topics:  49
['proces termoselektivní',
 'bubble policy [povolení obchodovatelná emisní]',
 'oceanografie v ústí řeky',
 'salinizace',
 'bastardizace fauny',
 'porost manglový pobřežní',
 'resuspenzace',
 'petrochemikálie',
 'les smíšený s mlázím',
 'bastardizace flóry',
 'flokuace',
 'remobilizace látky znečišťující',
 'perifyton',
 'vitirifikace',
 'nemoc Šagasova',
 'rostlina jevnosnubná',
 'sloučenina alkylická',
 'fluidika',
 'látka lipofilická',
 'článkovec',
 'embryogenéza',
 'perchloretylen',
 'faktor geogenní',
 'lov ryb do sítí tenatových',
 'lože fluidizované',
 'edafologie',
 'vináza',
 'dezintegrátor [šrédr]',
 'uhlovodík chlorfluorovaný',
 'odkrývka',
 'test teratogeneze',
 'látka xenobiotická',
 'oblast solitvorná',
 'uhlovodík chlorfluorovaný částečně halogenovaný',
 'teratogeneze',
 'chlorfenol',
 'reflektometrie',
 'hmyz [Heteroctera]',
 'olfaktometrie',
 'chloretylen',
 'pyralen',
 'metanizace',
 'Artropoda',
 'bezobratlí [Coelenterate]',
 'mycety',
 'halofor

In [24]:
# create new lists which don't containt topics with non-existent words

# load list with excluded topics
with open(excluded_filename, 'rt', encoding='utf8') as fp:
    tmp_topics = fp.readlines()
excluded_set = set()
for topic in tmp_topics:
    excluded_set.add(topic.strip())

with open(nolemma_list_file, 'rt', encoding='utf8') as fp:
        topics_file = fp.readlines()

topics_final = []
topics_lemma = []

excluded = 0
for topic in topics_file:
    topic_orig = topic.strip()
    if topic_orig in excluded_set:
        print("=== EXCLUDE ", topic_orig)
        excluded += 1
        continue
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []

    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics_lemma.append("_".join(lemmas))
    topics_final.append(topic_orig)

    print(topic_orig, ' --- ', "_".join(lemmas))

print('Excluded: ', excluded)
print('Final topics: ', len(topics_lemma), len(topics_final))
    
with open(lemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_lemma in topics_lemma:
        f.write(topic_lemma + "\n")
with open(nolemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_final in topics_final:
        f.write(topic_final + "\n")  

rozvoj místní  ---  rozvoj_místní
obnova venkova  ---  obnova_venkov
řízení zdrojů vodních  ---  řízení_zdroj_vodní
úroveň vzdělání  ---  úroveň_vzdělání
vzdělání obecné  ---  vzdělání_obecný
násep  ---  násep
právo Společenství  ---  právo_společenství
dědictví kulturní  ---  dědictví_kulturní
půda zemědělská  ---  půda_zemědělský
krajina zemědělská  ---  krajina_zemědělský
výroba zemědělská  ---  výroba_zemědělský
rozvoj ekonomický  ---  rozvoj_ekonomický
řetězec potravní  ---  řetězec_potravní
průmysl potravinářský  ---  průmysl_potravinářský
kvalita potravin  ---  kvalita_potravina
oteplování globální  ---  oteplování_globálný
efekt skleníkový  ---  efekt_skleníkový
rozvoj průmyslový  ---  rozvoj_průmyslový
trh práce  ---  trh_práce
přístup na pozemek  ---  přístup_pozemek
slučování pozemků  ---  slučování_pozemek
plánování územní  ---  plánování_územný
architektura krajinná  ---  architektura_krajinný
plánování krajinné  ---  plánování_krajinný
využití půdy  ---  využití_půda
plán

hnojení půdy  ---  hnojení_půda
rozvoj prostředí životního  ---  rozvoj_prostředí_životní
zdroj půdní  ---  zdroj_půdní
management  ---  management
politika  ---  politika
těžba dřeva  ---  těžba_dřevo
rekultivace půdy  ---  rekultivace_půda
systém dopravní  ---  systém_dopravní
salinita půdy  ---  salinita_půda
salinita vody  ---  salinita_voda
plánování rozvoje  ---  plánování_rozvoj
plánování dopravy  ---  plánování_doprava
zařízení zemědělské  ---  zařízení_zemědělský
zvíře hospodářské  ---  zvíře_hospodářský
legislativa k průmyslu  ---  legislativa_k_průmysl
pomoc finanční  ---  pomoc_finanční
kvalita prostředí životního  ---  kvalita_prostředí_životní
vlhkost půdní  ---  vlhkost_půdní
hospodářství zemědělské  ---  hospodářství_zemědělský
farma mléčná  ---  farma_mléčný
průmysl elektrotechnický  ---  průmysl_elektrotechnický
konzervace potravin  ---  konzervace_potravina
oblast spádová  ---  oblast_spádový
péče sociální  ---  péče_sociální
prostředí životní  ---  prostředí_životní

podnikání  ---  podnikání
stavby veřejné  ---  stavba_veřejný
bezpečnost ekologická  ---  bezpečnost_ekologický
trh peněžní  ---  trh_peněžní
činnost hospodářská  ---  činnost_hospodářský
činnost průmyslová  ---  činnost_průmyslový
trend populační  ---  trend_populační
potřeby bydlení  ---  potřeba_bydlení
politika výrobní  ---  politika_výrobní
region biogeografický  ---  region_biogeografický
systém komunikační  ---  systém_komunikační
pozemek stavební  ---  pozemek_stavebný
trh finanční  ---  trh_finanční
proces společenský  ---  proces_společenský
účinnost ekologická  ---  účinnost_ekologický
technologie energetická  ---  technologie_energetický
typ půdní  ---  typ_půdní
diverzita biologická  ---  diverzita_biologický
náhrada finanční  ---  náhrada_finanční
asanace části městské  ---  asanace_část_městský
průmysl chladírenský  ---  průmysl_chladírenský
znovuvyužití vody  ---  znovuvyužití_voda
síly ozbrojené  ---  síla_ozbrojený
škůdce v zemědělství  ---  škůdce_v_zemědělství
oblas

instituce veřejná  ---  instituce_veřejný
průmysl uhelný  ---  průmysl_uhelný
kal průmyslový  ---  kat_průmyslový
společnost  ---  společnost
analýza společnosti  ---  analýza_společnost
zařízení kulturní  ---  zařízení_kulturní
region  ---  region
emise znečišťující vodu  ---  emise_znečišťující_voda
plán ekonomický  ---  plán_ekonomický
ochrana civilní  ---  ochrana_civilní
půdoznalství  ---  půdoznalství
kvalita akustická  ---  kvalita_akustický
kritérium prostředí životního  ---  kritérium_prostředí_životní
znečištění fyzikální  ---  znečištění_fyzikální
systém daňový  ---  systém_daňový
zóna pobřežní  ---  zóna_pobřežní
rodičovství plánované  ---  rodičovství_plánovaný
les v oblasti Středozemního moře  ---  les_v_oblast_středozemní_moře
vnímání ekologické  ---  vnímání_ekologický
život studentský  ---  život_studentský
voda termální  ---  voda_termální
fond prostředí životního  ---  fond_prostředí_životní
plánování staveb  ---  plánování_stavba
návrh zákona pozměňovací  ---  návrh

statistika vod  ---  statistika_voda
proces bělící  ---  proces_bělící
podnik ekologický  ---  podnik_ekologický
boj ekologický  ---  boj_ekologický
hodnocení technologie  ---  hodnocení_technologie
zákon o vztazích sousedských  ---  zákon_o_vztah_sousedský
riziko přírodní  ---  riziko_přírodní
ochrana před povodněmi  ---  ochrana_povodeň
proces chemický  ---  proces_chemický
politika mzdová  ---  politika_mzdový
využití dehtu  ---  využití_dehet
zlepšení dat obrazových  ---  zlepšení_data_obrazův
energie větrná  ---  energie_větrný
plyn zemní  ---  plyn_zemní
systém kardiovaskulární  ---  systém_kardiovaskulární
změna hodnoty  ---  změna_hodnota
bilance (rozvaha) ekonomická  ---  bilance_rozvaha_ekonomický
malířství  ---  malířství
technologie vhodná  ---  technologie_vhodný
ochrana včel  ---  ochrana_včet
požár lesní  ---  požár_lesní
orgán sluchový  ---  orgán_sluchový
statistika prostředí životního  ---  statistika_prostředí_životní
les listnatý  ---  les_listnatý
výzkum vlivů  ---

...

cirkulace vody v oceánu  ---  cirkulace_voda_v_oceán
křižovatka  ---  křižovatka
vazba dusíku biologická  ---  vazb_dusík_biologický
technika přístrojová  ---  technika_přístrojový
=== EXCLUDE  vitirifikace
hladomor  ---  hladomor
dvůr soudní  ---  dvůr_soudní
zařízení elektrotechnické  ---  zařízení_elektrotechnický
emancipace  ---  emancipace
kal odvodněný  ---  kat_odvodněný
stav odpadů  ---  stav_odpad
rozpočet  ---  rozpočet
forma organizací právní  ---  forma_organizace_právní
experiment dlouhodobý  ---  experiment_dlouhodobý
fixace dusíku  ---  fixace_dusík
obydlí  ---  obydlit
poplatek licenční  ---  poplatek_licenční
mělčina bahnitá  ---  mělčina_bahnitý
záření alfa  ---  záření_alfa
zařízení pro záznam dat  ---  zařízení_záznam_datum
rozpoznání vzorce  ---  rozpoznání_vzorka
nařízení  ---  nařízení
prvky skupiny V  ---  prvek_skupina_v
stěna norná  ---  stěna_norný
výzbroj vojenská  ---  výzbroj_vojenský
oxygenace  ---  oxygenace
vztahy měnové  ---  vztah_měnový
vymáhání práv

kompresor  ---  kompresor
experiment  ---  experiment
bavlna  ---  bavlna
adsorpce chemická  ---  adsorpce_chemický
vodivost  ---  vodivost
oxidace  ---  oxidace
rychlost  ---  rychlost
vepřín  ---  vepřín
minimalizace odpadů  ---  minimalizace_odpad
normalizace  ---  normalizace
technika rozmnožování umělého  ---  technika_rozmnožování_umělý
těžba ze dna mořského  ---  těžba_dno_mořský
prémie za snížení znečištění  ---  prémie_snížení_znečištění
produkt hrubý domácí  ---  produkt_hrubý_domácí
zatížení náhlé  ---  zatížený_náhlý
úprava kovů povrchová  ---  úprava_koev_povrchovat
astronautika  ---  astronautika
odpadky z lodí  ---  odpadek_z_loď
kejda kapalná  ---  kejda_kapalný
kov  ---  kov
právo průchodu přes majetek cizí  ---  právo_průchod_majetek_cizí
věž chladicí  ---  věž_chladice
mýdlo  ---  mýdlo
atlas rostlin  ---  atlas_rostlina
olej chladicí  ---  olej_chladice
plyn výfukový z vozidel  ---  plyn_výfukový_z_vozidlo
koncepce právní neomezená  ---  koncepce_právní_omezený
vápn

čin protiprávní  ---  čin_protiprávní
sklo  ---  sklo
azbest rozprašovaný  ---  azbest_rozprašovaný
prosvětlování křovin  ---  prosvětlování_křovina
jednotka na třídění odpadů  ---  jednotka_třídění_odpad
čas  ---  čas
kyselina nukleová  ---  kyselina_nukleový
prevence proti požáru  ---  prevence_požár
informace referenční  ---  informace_referenční
sbor zákonodárný  ---  sbor_zákonodárný
hnutí ekologické  ---  hnutí_ekologický
teplota  ---  teplota
dovoz  ---  dovoz
mikroklimatologie  ---  mikroklimatologie
farmakokinetika  ---  farmakokinetika
vrt ropný  ---  vrt_ropný
látka  netěkavá  ---  látka_těkavat
pesticid reziduální  ---  pesticid_reziduální
sdružení  ---  sdružení
obsah hliníku  ---  obsah_hliník
práce diplomová/ dizertační  ---  práce_diplomový_dizertační
čištění měst  ---  čištění_město
bróm  ---  bróm
divočina  ---  divočina
zdroj přímý  ---  zdroj_přímý
odpady pryžové  ---  odpad_pryžový
schopnost substituce  ---  schopnost_substituce
sankce  ---  sankce
kov barevný  ---

zaprášení plic  ---  zaprášený_plíce
hlídka pobřežní  ---  hlídka_pobřežní
oblast tření ryb  ---  oblast_tření_ryba
alkohol  ---  alkohol
prostředek bělicí  ---  prostředek_bělice
=== EXCLUDE  faktor geogenní
sůl  ---  sůt
slovník  ---  slovník
zplyňování uhlí  ---  zplyňování_uhlí
šrot železný  ---  šrot_železný
poplatek  ---  poplatek
nádrž vyhnívací  ---  nádrž_vyhnívací
salmonella  ---  salmonella
vanad  ---  vanad
zvuk  ---  zvuk
=== EXCLUDE  lov ryb do sítí tenatových
třídění  ---  třídění
antibiotikum  ---  antibiotikum
flotace  ---  flotace
leukémie  ---  leukémie
limit vystavení přípustného  ---  limit_vystavený_přípustný
látka adsorbující ropu  ---  látka_adsorbující_ropa
externalita  ---  externalita
světlo  ---  světlo
ještěrka  ---  ještěrka
úřad  ---  úřad
vyplavování  ---  vyplavování
tanin  ---  tanina
míchání  ---  míchání
had rodu Colubridae  ---  had_rod_colubridae
budova v místě vyvýšeném  ---  budova_v_místo_vyvýšený
vápno  ---  vápno
látka odbouratelná obtížně  --

jed  ---  jed
činidlo změkčovací  ---  činidnout_změkčovací
byrokratizace  ---  byrokratizace
dibenzodioxin polychlorovaný  ---  dibenzodioxin_polychlorovaný
hematologie  ---  hematologie
škvor  ---  škvor
saprofyt  ---  saprofyt
stěhování  ---  stěhování
bifenyl polybromovaný  ---  bifenýt_polybromovaný
bifenyl polychlorovaný  ---  bifenýt_polychlorovaný
bibliografie  ---  bibliografie
prostředek konzervační  ---  prostředek_konzervační
mělčina příbřežní  ---  mělčina_příbřežní
odpisy  ---  odpisa
zápach  ---  zápach
statistika  ---  statistika
algicid  ---  algicid
údaje georeferenční  ---  údaj_georeferenční
DDT  ---  ddt
encyklopedie  ---  encyklopedie
dendrochronologie  ---  dendrochronologie
=== EXCLUDE  hmyz [Heteroctera]
kolorimetrie  ---  kolorimetrie
kyanat  ---  kyanat
vulkanizace  ---  vulkanizace
purin  ---  purin
norma  ---  norma
mletí  ---  mletí
izomer  ---  izomera
aktinium  ---  aktinium
mutagen  ---  mutagen
=== EXCLUDE  olfaktometrie
plazma zárodečná  ---  plazma_z

In [25]:
# load the lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

#try if the topic-lemma words exist in the model     
new_vectors = []
new_words = []
for i,topic in enumerate(topics):
    #print(i, topic)
    topic = topic.strip()
    topic_words = topic.split('_')
    #if len(topic_words)<2:
    #    continue
    tmp_vectors = []
    for word in topic_words:
        
        try:
            tmp_vectors.append(model[word])
        except: 
            print('\nNOT in vocabulary: ', word)
            vector = get_vector(word)
            if vector != []:
                tmp_vectors.append(vector)
                if word not in new_words:
                    print('append ', word)
                    new_vectors.append(vector)
                    new_words.append(word)   
            else:
                raise Warning(f"XXX {word} {topic}")
                print("XXX", word, topic) 
    #if word not in new_words:
    #    new_vectors.append(list(np.average(tmp_vectors, axis=0)))
    #    new_words.append(word)        
    if len(tmp_vectors) > 1:
        if topic not in new_words:
   
            print('append ', topic)
            new_vectors.append(list(np.average(tmp_vectors, axis=0)))
            new_words.append(topic)
        else:
            print('doubled TOPIC: ', topic)    
    else:
        print('--- only one word, not a new topic>', topic)

append  rozvoj_místní
append  obnova_venkov
append  řízení_zdroj_vodní
append  úroveň_vzdělání
append  vzdělání_obecný
--- only one word, not a new topic> násep
append  právo_společenství
append  dědictví_kulturní
append  půda_zemědělský
append  krajina_zemědělský
append  výroba_zemědělský
append  rozvoj_ekonomický
append  řetězec_potravní
append  průmysl_potravinářský
append  kvalita_potravina
append  oteplování_globálný
append  efekt_skleníkový
append  rozvoj_průmyslový
append  trh_práce
append  přístup_pozemek
append  slučování_pozemek
append  plánování_územný
append  architektura_krajinný
append  plánování_krajinný
append  využití_půda
append  plánování_využití_půda
append  prostředí_životní_mořský
append  ochrana_příroda
doubled TOPIC:  ochrana_příroda
append  zemědělství_ekologický
append  plánování_prostorový
append  vzdělání_základní
append  průmysl_automobilový
append  kvalita_život
append  rozvoj_regionální
append  hospodaření_zdroj
append  obyvatelstvo_venkovský
append  vzdě

append  prostředí_geofyzikální
append  voda_říční
append  šetření_s_voda
append  financování_veřejný
append  bydlení_nájemné
doubled TOPIC:  obdělávání_půda
--- only one word, not a new topic> zpracování
append  efekt_mikroklimatický
append  řízení_integrovaný
append  funkce_půda
append  zemědělství_smíšený
append  zeleně_městský
append  voda_surový
append  les_původní
append  úroveň_komunální
append  průmysl_surovina_nerostný
append  řízení_správný
append  cesta_přístupový
append  právo_soukromý
append  právo_občanský
append  chemie_půda
append  věda_fyzikální
append  nauka_o_voda
append  hustota_obyvatelstvo
append  úhory_v_důsledek_činnost_průmyslový_předchozí
append  politika_vládní
append  spolupráce_rozvojový
--- only one word, not a new topic> organizace
append  struktura_průmysl
append  struktura_ekonomický
append  zakázka_veřejný
append  osoba_starý
append  znečištění_pobřeží
append  znečištění_voda_sladký
append  chov_drůbež
append  únosnost_půda
append  trh_environmentální
a

/home/dzon/env/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


doubled TOPIC:  ochrana_les
append  znečištění_průmyslový
append  oblast_polární
append  přenos_technologie
append  surovina_nerostný
append  hygiený_prostředí_životní
append  zvuk_vydávaný_zvíře
append  les_vysoký
append  nebezpečí_prostředí_životní
append  budova_vládní
append  oblast_rizikový
append  koncentrace_sídlo
append  hodnocení_technologie
append  skladiště_cisterna
--- only one word, not a new topic> souvrátit
append  vliv_člověk
append  politika_v_oblast_technika_sdělovací
append  vliv_genetický
append  palivo_motorový
append  nástroj_ekonomický
append  praxe_zemědělský
append  podnebí_rovníkový
append  stav_technika
--- only one word, not a new topic> cena
append  obchod_hospodářský
append  síť_kanalizační
append  nauka_o_prostředí_životní
--- only one word, not a new topic> zdraví
append  zvíře_domestikovaný
append  osada_přechodný
append  park_národní
append  organizace_práce
append  stimul_ekonomický
--- only one word, not a new topic> pojišťovna
append  statistika_reg

append  problematika_ekonomika_prostředí_životní
append  síť_prostorově_rozsáhlý
append  ochrana_povodí
append  průmysl_materiál_syntetický
append  úprava_povrchový
append  směrnice_o_čistota_voda
append  průmysl_vlákno_syntetický
append  ekosystém_mokřadní
append  systém_informační_o_prostředí_životní
append  hospodářství_odpadový_duální
append  nařízení_ochrana_prostředí_životní
append  průmysl_hmota_stavební
append  část_park_centrální
append  technologie_energie_sluneční
append  technika_zpracování_obraz_digitální
append  podmínka_povětrnostní
append  evropský_měnový_fond
append  trh_evropský_vnitřní
doubled TOPIC:  odtok_povrchový
append  měření_znečištění_fyzikální
append  řízení_trestní
append  ochrana_voda_mořský
append  úpravna_voda
append  vlastnictví_cesta_vodní_veřejný
append  evropský_měnový_systém
append  monitorování_počasí
append  standard_bezpečnostní_budova
append  znečištění_zapříčiněný_vozidlo_motorový
append  program_rozvoj_město_akční
append  účetnictví_ekologický

append  využití_materiál_opětný
append  znečištění_biologický
append  skupina_cílový
--- only one word, not a new topic> mikroelektronika
append  faktor_emisní
append  demolice_budova
--- only one word, not a new topic> nákup
append  znečištění_tepelný
append  obnova_energie
append  doprava_letecký
--- only one word, not a new topic> spad
append  analýza_chemický
append  vrstva_vodonosný
--- only one word, not a new topic> onkologie
--- only one word, not a new topic> píska
append  výroba_nástroj
append  model_atmosféra
--- only one word, not a new topic> biologie
append  oxid_titaničitý
--- only one word, not a new topic> geomorfologie
append  oblast_těžba_důlní
append  materiál_elektronický
--- only one word, not a new topic> mikropočítač
append  program_rozšíření
append  oblast_turistický
append  antarktický_oceán
append  analýza_fyzikálně_chemický
append  monitorování_záření
append  oxid_dusičitý
--- only one word, not a new topic> folklór
append  snižování_emise
--- only one word,

append  nádrž_palivový
append  reakce_chemický
append  výstavba_cesta_dopravní
append  odpad_z_proces_čištění
append  emise_radioaktivní
append  spotřeba_ropa
append  migrace_vnitrostátní
append  půjčování_dokument
append  fyziologie_ekologický
append  koryto_říční
--- only one word, not a new topic> elektrotechnika
append  znečištění_radioaktivní
append  úřad_federální
append  odpad_kapalný
append  konflikt_mezinárodní
append  produkt_vedlejší
--- only one word, not a new topic> výrobek
--- only one word, not a new topic> polotovar
append  dohoda_multilaterální
append  monitorování_samočinný
append  potravina_zemědělský
--- only one word, not a new topic> potravina
append  pěstování_dřevina
append  listina_právní
--- only one word, not a new topic> okres
append  čistota_hygieno
--- only one word, not a new topic> hora
--- only one word, not a new topic> genetika
append  jednotka_turistický
append  šum_impulsní
append  sankce_správní
append  emise_látka_znečišťující
append  zdroj_dopad

--- only one word, not a new topic> klimatologie
append  manipulace_reprodukční
append  měření_zvuk
append  zařízení_akumulak_energie_elektrický

NOT in vocabulary:  iucn
doubled TOPIC:  druh_ohrožený_iucn
append  nařízení_úřední
append  látka_znečištěný
append  kontrola_hluk_z_doprava
--- only one word, not a new topic> bílkovina
doubled TOPIC:  řízení_trestní

NOT in vocabulary:  cytotoxicita
append  cytotoxicita
--- only one word, not a new topic> cytotoxicita
--- only one word, not a new topic> šlechtění
doubled TOPIC:  vlna_mořský
append  odpad_domovní
append  ukládání_odpad_v_moře_volný
append  údaj_provozní

NOT in vocabulary:  limnologie
append  limnologie
--- only one word, not a new topic> limnologie
append  baterie_knoflíkový
--- only one word, not a new topic> korek
append  látka_plynný_znečišťující_ovzduší
--- only one word, not a new topic> odstřeďování
--- only one word, not a new topic> stanice
--- only one word, not a new topic> kyselina
append  tanker_ropný
append  ob

append  pěnotvorný
append  prostředek_pěnotvorný
--- only one word, not a new topic> fytosociologie
--- only one word, not a new topic> kartografie
append  odstraňování_látka_znečišťující
append  stanovení_cíl
append  výměna_informace_ohledně_riziko
--- only one word, not a new topic> rastr
doubled TOPIC:  právo_občanský
--- only one word, not a new topic> AIDS

NOT in vocabulary:  paleoekologie
append  paleoekologie
--- only one word, not a new topic> paleoekologie
--- only one word, not a new topic> tabák
append  ubytování_nouzový
append  schopnost_asimilace_odpad
append  indikátor_znečištění
--- only one word, not a new topic> septik
--- only one word, not a new topic> databáze
append  výška_komín
--- only one word, not a new topic> slinování
append  spalování_odpad
append  testování_toxikologický
append  oblast_rekreační_břeh_mořský
append  zbytek_zpracování
append  strom_širokolistý
append  zneškodňování_odpad_z_loď
append  potenciál_recyklační
append  území_registrovaný
append  d

--- only one word, not a new topic> étro
--- only one word, not a new topic> záliv
--- only one word, not a new topic> petrolej
append  spalování_společný
--- only one word, not a new topic> delta
--- only one word, not a new topic> dávkování

NOT in vocabulary:  pentachlorfenol
append  pentachlorfenol
--- only one word, not a new topic> pentachlorfenol
doubled TOPIC:  třídění_u_zdroj
append  práce_školní
--- only one word, not a new topic> odstraňování

NOT in vocabulary:  onchocerkóza
append  onchocerkóza
--- only one word, not a new topic> onchocerkóza
--- only one word, not a new topic> emulze
append  výška_komín_nadměrný
--- only one word, not a new topic> ontogenézt
--- only one word, not a new topic> index
append  ostrost_sluch
append  stezka_turistický
--- only one word, not a new topic> zárodek
--- only one word, not a new topic> šváb
--- only one word, not a new topic> skládkování
--- only one word, not a new topic> přehrada
append  látka_impregnační
--- only one word, not a 

In [26]:
# add topics to model
model.add(new_words,new_vectors)

In [27]:
# save the model with new topics
model.save_word2vec_format(excluded_model_name, binary=True)
#reload the model  otherwise cannot find similarities fo newly added words
model = KeyedVectors.load_word2vec_format(excluded_model_name, binary=True)

In [28]:
# try the model
model.similar_by_vector('plavba_vnitrozemský')

[('plavba', 0.9420293569564819),
 ('plavba_námořní', 0.9187299013137817),
 ('plavba_zábavní', 0.9030261039733887),
 ('plavba_výletní', 0.8916735649108887),
 ('loď_rybářský', 0.7599372267723083),
 ('vnitrozemský', 0.7412785291671753),
 ('loď_cisternový', 0.7083302736282349),
 ('doprava_vodní_vnitrozemský', 0.701938271522522),
 ('loď', 0.7014586925506592),
 ('stavitelství_loď', 0.7012679576873779)]

In [29]:
# find most similar to topics to terms in the new model

# load the new lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

for i, topic in enumerate(topics):
    topic = topic.strip()
    print('\n-----\n', topic)
    try:
        print(i, model.similar_by_vector(topic))
    except:
        print(topic, 'NOT in model')


-----
 rozvoj_místní
0 [('rozvoj_regionální', 0.9031485915184021), ('rozvoj_ekologický', 0.8777536153793335), ('rozvoj_technologický', 0.8745042085647583), ('rozvoj_ekonomický', 0.8729321956634521), ('rozvoj_průmyslový', 0.8727995157241821), ('rozvoj', 0.8685276508331299), ('program_rozvoj_město_akční', 0.8668836951255798), ('rozvoj_udržitelný', 0.860724925994873), ('místní', 0.8601834177970886), ('rozvoj_urbanistický', 0.859067976474762)]

-----
 obnova_venkov
1 [('obnova', 0.830689549446106), ('venkov', 0.8095443844795227), ('obnova_zařízení', 0.8082926869392395), ('obnova_stanoviště', 0.7957338690757751), ('obnova_městský', 0.7859998941421509), ('obnova_vegetace', 0.7531415820121765), ('obnova_energie', 0.7422579526901245), ('obnova_flóra', 0.7217881679534912), ('regenerace_krajina', 0.7063193321228027), ('obnova_místo_těžba', 0.6921186447143555)]

-----
 řízení_zdroj_vodní
2 [('ochrana_zdroj_vodní', 0.9365036487579346), ('legislativa_zdroj_vodní', 0.931503176689148), ('rozvoj_zdro

21 [('územný', 0.9572497606277466), ('územní', 0.9572497606277466), ('projekt_územný', 0.9373807907104492), ('plán_územný', 0.931808590888977), ('společenství_územný', 0.9200952053070068), ('rozhodnutí_územný', 0.9183609485626221), ('rozvoj_územný', 0.9069401025772095), ('model_územní_digitální', 0.8506943583488464), ('plánování_regionální', 0.8142982125282288), ('plánování_ekonomický', 0.7993893027305603)]

-----
 architektura_krajinný
22 [('krajinný', 0.9515780210494995), ('plánování_krajinný', 0.9209429621696472), ('ekologie_krajinný', 0.8769493103027344), ('prvek_krajinný', 0.8688483834266663), ('oblast_chráněný_krajinný', 0.8571244478225708), ('krajina_zemědělský', 0.7594929337501526), ('krajina_terasovitý', 0.7550214529037476), ('krajinářský', 0.7524529695510864), ('krajina_městský', 0.7468706965446472), ('krajina_chráněný', 0.7413658499717712)]

-----
 plánování_krajinný
23 [('krajinný', 0.9319091439247131), ('architektura_krajinný', 0.9209429025650024), ('oblast_chráněný_krajin

41 [('rozvoj', 0.9441356658935547), ('rozvoj_ekonomický', 0.9425983428955078), ('rozvoj_ekologický', 0.939881443977356), ('rozvoj_technologický', 0.9235219359397888), ('rozvoj_průmyslový', 0.9167841672897339), ('plánování_rozvoj', 0.908583402633667), ('rozvoj_společnost', 0.9076065421104431), ('rozvoj_regionální', 0.9075832366943359), ('rozvoj_urbanistický', 0.9036427736282349), ('rozvoj_urbanistický_plánovaný', 0.8861017823219299)]

-----
 urbanismus
42 [('urbanista', 0.7960213422775269), ('urbanistický', 0.7750227451324463), ('rozvoj_urbanistický', 0.706862211227417), ('studie_urbanistický', 0.6781168580055237), ('architektura_krajinný', 0.6669037938117981), ('urbanizace', 0.6662325859069824), ('architektura', 0.661422848701477), ('rozvoj_urbanistický_plánovaný', 0.6554299592971802), ('návrh_urbanistický', 0.6419765949249268), ('suburbanizace', 0.6281270980834961)]

-----
 plocha_vodní
43 [('prostředí_vodní', 0.8503925800323486), ('vodní', 0.8398171663284302), ('hladina_vodní', 0.836

72 [('zdroj_voda_sladký', 0.9180067181587219), ('zdroj_voda_pitný_město', 0.9066411256790161), ('zdroj_vypouštění_voda_odpadní_přímý', 0.8635270595550537), ('zdroj_znečištění', 0.8588306307792664), ('statistika_voda', 0.852031946182251), ('využití_voda', 0.8480139970779419), ('spotřeba_voda', 0.8470696210861206), ('voda', 0.8442868590354919), ('voda_použitý', 0.8414120674133301), ('potřeba_voda', 0.8411520719528198)]

-----
 management_zdroj_přírodní
73 [('zdroj_přírodní', 0.9338608980178833), ('zachování_zdroj_přírodní', 0.9018721580505371), ('zdroj_biologický', 0.8770085573196411), ('zdroj_rostlinný', 0.8759108185768127), ('zdroj_informační', 0.869985818862915), ('zdroj_živočišný', 0.8617827892303467), ('zdroj_obnovitelný', 0.8607458472251892), ('zabezpečení_zdroj_surovinový', 0.854863166809082), ('rozvoj_zdroj_vodní', 0.8542046546936035), ('zdroj_znečištění', 0.8513317704200745)]

-----
 ochrana_oblast_přírodní
74 [('oblast_přírodní', 0.9206205606460571), ('ochrana_oblast_horský', 0

99 [('rozvoj_urbanistický_plánovaný', 0.961353063583374), ('rozvoj_ekonomický', 0.9207128882408142), ('rozvoj_technologický', 0.9171364307403564), ('rozvoj', 0.9154572486877441), ('rozvoj_průmyslový', 0.9148131608963013), ('rozvoj_ekologický', 0.9056191444396973), ('plánování_rozvoj', 0.904679536819458), ('rozvoj_udržitelný', 0.9036427736282349), ('rozvoj_regionální', 0.9000418782234192), ('rozvoj_územný', 0.8968020081520081)]

-----
 politika_městský
100 [('prostředí_městský', 0.8957603573799133), ('městský', 0.8849056363105774), ('obyvatelstvo_městský', 0.8836362361907959), ('zařízení_městský', 0.8698514103889465), ('oblast_městský', 0.8516359925270081), ('centrum_městský', 0.843177318572998), ('aglomerace_městský', 0.841834306716919), ('správa_městský', 0.8386691808700562), ('doprava_městský', 0.8324416875839233), ('politika_kulturní', 0.831396222114563)]

-----
 obyvatelstvo_městský
101 [('městský', 0.9065070152282715), ('prostředí_městský', 0.9038696885108948), ('politika_městský'

...

5020 [('nehořlavost', 0.8372422456741333), ('hořlavý', 0.6549934148788452), ('látka_hořlavat', 0.64925616979599), ('hořlava', 0.6378276944160461), ('hořlavat', 0.6378204226493835), ('hořlavina', 0.6313776969909668), ('nasákavost', 0.6117380261421204), ('nehořlavý', 0.6067219972610474), ('hořlavá', 0.5644747614860535), ('látka_ohnivzdorný', 0.5419838428497314)]

-----
 hmota_pohonný
5021 [('stanice_čerpací_hmota_pohonný', 0.8768244981765747), ('pohonný', 0.8457140922546387), ('hmota_nátěrový', 0.8263211846351624), ('hmota', 0.818554699420929), ('hmota_stavebný', 0.7911851406097412), ('průmysl_hmota_stavební', 0.7699928879737854), ('voda_hmota', 0.7415040135383606), ('přenos_hmota_fyzika', 0.6970400810241699), ('částice_tuhý', 0.6680198311805725), ('palivo_motorový', 0.6656304597854614)]

-----
 biofyzika
5022 [('biofyzikální', 0.7258242964744568), ('fyzika', 0.6776140928268433), ('fyzika_radiační', 0.6682733297348022), ('biochemie', 0.6632794141769409), ('fyziologie', 0.641396164894104)

5049 [('houba', 0.905053436756134), ('stopkovýtrusný', 0.8040002584457397), ('houba_mořský', 0.799640953540802), ('vřeckovýtrusný', 0.756006121635437), ('houb', 0.743645191192627), ('plodnice', 0.65868079662323), ('houbový', 0.6586227416992188), ('výtrusný', 0.6584528684616089), ('houbovec', 0.6579062938690186), ('výtrus', 0.6440874934196472)]

-----
 ropa
5050 [('rafinérie_ropa', 0.9031896591186523), ('spotřeba_ropa', 0.8782033920288086), ('rafinace_ropa', 0.8507595062255859), ('geologie_ropa', 0.8470525145530701), ('těžba_ropa', 0.8307979106903076), ('průzkum_ložisko_ropa', 0.7728230953216553), ('proces_zpracování_ropa', 0.7661454081535339), ('těžba_ropa_v_moře_volný', 0.7348622679710388), ('látka_adsorbující_ropa', 0.7232526540756226), ('energie_z_ropa', 0.6977092027664185)]

-----
 normalizace
5051 [('normalizace_mezinárodní', 0.8332658410072327), ('normalizační', 0.7613413333892822), ('normalizovaný', 0.6663042306900024), ('formalizace', 0.606802225112915), ('normalizovat', 0.6038

5079 [('drcení', 0.7002986073493958), ('pražení', 0.6391596794128418), ('mlece', 0.6334266662597656), ('šrotování', 0.628993809223175), ('pěchování', 0.6217010617256165), ('spaření', 0.6104158163070679), ('mlecí', 0.6052820682525635), ('spékání', 0.5909249186515808), ('strouhání', 0.587356448173523), ('míchání', 0.5870416760444641)]

-----
 izomera
5080 [('izomer', 0.8941174149513245), ('izomerie', 0.8615293502807617), ('enantiomera', 0.7227801084518433), ('hydroxýt', 0.6562638282775879), ('monomera', 0.639030933380127), ('sloučenina_heterocyklický', 0.6336157917976379), ('sloučenina_alifatický', 0.6199369430541992), ('dimera', 0.6143761873245239), ('ligand', 0.6087746620178223), ('sloučenina_alicyklický', 0.6082000732421875)]

-----
 aktinium
5081 [('hafnium', 0.716792106628418), ('aktinoid', 0.6455890536308289), ('germanium', 0.6304836273193359), ('gadolinium', 0.6287221312522888), ('yttrium', 0.605554461479187), ('curium', 0.5994151830673218), ('baryum', 0.5942023992538452), ('europ

5109 [('cirkulace_voda_v_oceán', 0.8768954873085022), ('čištění_voda_odpadní_u_zdroj', 0.8736773729324341), ('voda_odpadní_z_činnost_obchodní', 0.8569705486297607), ('voda_odpadní_z_lisovna_olej_olivový', 0.8529147505760193), ('voda_v_půda', 0.8283361196517944), ('ukládání_odpad_v_moře_volný', 0.8277029991149902), ('zneškodňování_odpad_v_půda', 0.8069162964820862), ('sediment_v_moře_volný', 0.8042699694633484), ('čištění_voda_odpadní', 0.7990108132362366), ('chemikálie_v_prostředí_životní', 0.7988032102584839)]

-----
 filtrace_biologický
5110 [('filtrace', 0.8988007307052612), ('čištění_filtrace_půdní', 0.8471304178237915), ('filtr_biologický', 0.838851273059845), ('filtrace_data_obrazův', 0.8126750588417053), ('čištění_biologický', 0.7664231657981873), ('filtrační', 0.7464941143989563), ('sterilizace_biologický', 0.7350273132324219), ('odbouratelnost_biologický', 0.733917236328125), ('znečištění_biologický', 0.7336630821228027), ('kontaminace_biologický', 0.7239803075790405)]

-----


5138 [('hnojení', 0.8625916242599487), ('přehnojení', 0.7602998614311218), ('hnojení_půda', 0.7515078783035278), ('spásání_pastvina_nadměrný', 0.7375349998474121), ('nadměrný', 0.7348330616950989), ('využívání_nadměrný', 0.7291659116744995), ('přihnojení', 0.7108974456787109), ('hnojený', 0.6794421672821045), ('hnojivo_zelený', 0.6782850027084351), ('pěstování_rostlina_selektivní', 0.6782104969024658)]

-----
 fytomaso
5139 [('biomasa', 0.6784824132919312), ('meziplodina', 0.6066264510154724), ('siláž', 0.5920896530151367), ('plodina', 0.5825308561325073), ('pícnina', 0.5685982704162598), ('luskovina', 0.5667001008987427), ('plodina_kořenový', 0.5648143887519836), ('bioplyn', 0.561079740524292), ('olejnina', 0.5592484474182129), ('hnojení_půda', 0.5526430606842041)]

-----
 ploutvonožec
5140 [('hlavonožec', 0.7203304767608643), ('kytovec', 0.6791038513183594), ('savec_suchozemský', 0.6644123792648315), ('dlouhoploutvý', 0.639777660369873), ('vačnatec', 0.633540153503418), ('chobotnatec